In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# New Section

In [ ]:
# Preprocessing steps (similar to Step 1)
# Drop non-beneficial columns
df = df.drop(columns=['EIN', 'NAME'])


In [ ]:
# Create bins for APPLICATION_TYPE and CLASSIFICATION
application_counts = df['APPLICATION_TYPE'].value_counts()
class_counts = df['CLASSIFICATION'].value_counts()

application_types_to_replace = list(application_counts[application_counts < 100].index)
classifications_to_replace = list(class_counts[class_counts < 100].index)

for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app, "Other")

for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls, "Other")

In [ ]:
# Encode categorical variables
df = pd.get_dummies(df)


In [ ]:
# Split data into features and target
X = df.drop(columns=['IS_SUCCESSFUL'])
y = df['IS_SUCCESSFUL']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [ ]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Define the neural network model with further optimizations
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])




In [ ]:
# Compile the model with a different optimizer
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model with more epochs
history = model.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_data=(X_test_scaled, y_test))


Epoch 1/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5826 - accuracy: 0.7132 - val_loss: 0.5631 - val_accuracy: 0.7199
Epoch 2/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5641 - accuracy: 0.7280 - val_loss: 0.5568 - val_accuracy: 0.7244
Epoch 3/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5601 - accuracy: 0.7293 - val_loss: 0.5577 - val_accuracy: 0.7222
Epoch 4/150
804/804 [==============================] - 3s 4ms/step - loss: 0.5569 - accuracy: 0.7305 - val_loss: 0.5556 - val_accuracy: 0.7213
Epoch 5/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5538 - accuracy: 0.7324 - val_loss: 0.5544 - val_accuracy: 0.7230
Epoch 6/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5536 - accuracy: 0.7309 - val_loss: 0.5585 - val_accuracy: 0.7215
Epoch 7/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5521 - accuracy: 0.7313 - val_loss: 0.5552 - val_accuracy: 0.7206

In [ ]:
# Evaluate the model
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 [==============================] - 0s 2ms/step - loss: 0.5540 - accuracy: 0.7247
Loss: 0.5539974570274353, Accuracy: 0.7246647477149963


In [ ]:
# Save the optimized model to HDF5 file
model.save("AlphabetSoupCharity_Optimization_Attempt3.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
